In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding("BAAI/bge-large-en-v1.5")

/Users/tanmaysingewar/Projects/ChunkSize/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
index_dir = 'countries/512-countries' 

In [3]:
from llama_index.core import StorageContext, load_index_from_storage
index = load_index_from_storage(storage_context=StorageContext.from_defaults(persist_dir=rf'index/{index_dir}'),embed_model=embed_model,llm=None)

In [4]:
retriever = index.as_retriever(similarity_top_k=5)

In [5]:
from llama_index.llms.openai import OpenAI
llm=OpenAI(llm="gpt-4-turbo")

In [6]:
from llama_index.core.chat_engine import ContextChatEngine
chat_engine = ContextChatEngine.from_defaults(llm=llm,retriever=retriever)

In [7]:
def get_answer(question):
    response = chat_engine.chat(str(question))
    return response.response

In [10]:
import time
import csv

with open('csv/countries-questions.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    csv_data = list(reader)

# Loop from 3 to 30 top_k
try:
    print(f" ----------------- Details ------------------ ")
    print(f"Model Name: gpt-4-turbo")
    print(f"Number of Questions: {len(csv_data) - 1}")
    print(f"Index: index/{index_dir}")
    print(f"--------------------------------------------- ")

    for row in csv_data:
        index = csv_data.index(row)
        if index == 0:
            csv_data[index].append(f"gpt-4-turbo")
            csv_data[index].append(f"Response time of gpt-4-turbo")
            continue

        print(f"-------------------------- Question No : {index} --------------------------")
        print(f"Question: {row[0]}")

        try:
            question_text = f"Question: {row[0]}\n"
            start_time = time.time()
            response_llm = get_answer(question_text)
            end_time = time.time()
            response_time = end_time - start_time
            print(f"LLM Response: {response_llm}")
            print(f"Response Time: {response_time:.2f} seconds")
            # Append the LLM response and the corresponding response time
            csv_data[index].append(response_llm)
            csv_data[index].append(f"{response_time:.2f} seconds")

            with open(f'csv/countries/gpt-4-turbo-response.csv', mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerows(csv_data)
        except Exception as e:
            print(f"Error: {e}")
            with open(f'csv/countries/gpt-4-turbo-response.csv', mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                csv_data[index].append("Error : Unable to retrieve LLM response")
                csv_data[index].append(f"Error : Unable to retrieve LLM response")
                writer.writerows(csv_data)
                
            continue

except Exception as e:
    print(f"Error: {e}")

 ----------------- Details ------------------ 
Model Name: gpt-4-turbo
Number of Questions: 100
Index: index/countries/512-countries
--------------------------------------------- 
-------------------------- Question No : 1 --------------------------
Question: What are some reasons cited for Afghanistan being among the least developed countries in the world?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


LLM Response: Some reasons cited for Afghanistan being among the least developed countries in the world include:

1. Rough physical geography
2. Landlocked status
3. Contemporary conflict
4. Political instability
Response Time: 3.97 seconds
-------------------------- Question No : 2 --------------------------
Question: Where was Djamel Laouisset's book "A Retrospective Study of the Algerian Iron and Steel Industry" published?
LLM Response: Djamel Laouisset's book "A Retrospective Study of the Algerian Iron and Steel Industry" was published in New York City by Nova Publishers.
Response Time: 1.56 seconds
-------------------------- Question No : 3 --------------------------
Question: Which countries are part of the Middle East region according to the document?
LLM Response: According to the document, the countries that are part of the Middle East region are:

1. Bahrain
2. Cyprus
3. Egypt
4. Iran
5. Iraq
6. Israeli Kurdistan
7. Israel
8. Jordan
9. Kuwait
10. Lebanon
11. Northern Cyprus
1